[Data Visualization](https://infovis.fh-potsdam.de/tutorials/) · FH Potsdam · Summer 2023

# Tutorial 5: Temporal analysis

The first four tutorials were meant to get you started, do a bit of data wrangling and try out interaction techniques. In the next five tutorials we will focus on particular data structures and the specific ways they are parsed, analyzed and visualized. We will start with temporal data.

We encounter time series data in pretty much every domain, from finance to weather, from public health to renewable energies. Visualizations of temporal data may represent recorded observations from the past and/or predicted developments for the future, which is why visual representations of temporal data are so important and interesting. Especially, in the context of the climate and corona crises we have been encountering many time series visualizations. 

This tutorial—just as the next ones—is structured into three basic phases: 1. Prepare, 2. Process, and 3. Present.


## 🛒 1. Prepare 

Before we are able to do anything, we need to include the libraries that we are working with (as always):

In [1]:
import pandas as pd
import altair as alt

### Parse dates and times

In its most basic form, time series data contains a quantitative measure that changes over time. To reference a time point we can use [Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html) of Pandas as the data type for temporal entities.

With **`to_datetime()`** you can create a Timestamp with a string containing a date and/or time. Pandas can infer the date and time from various date/time strings. Let's start with the present:


In [48]:
pd.to_datetime("today")

Timestamp('2023-06-01 19:35:11.049256')

We can pass a range of date formats and Pandas will guess which numbers refer to years, months, days, hours, etc.:

In [3]:
pd.to_datetime('2020-05-07 3pm')

Timestamp('2020-05-07 15:00:00')

✏️ *Enter different date or time formats and see how they are interpreted!*


When expressing dates and times in written language, there is an ambiguity between the order of different entities. The most frequent ambiguity concerns the order of days and months, as they are typically both expressed in double-digit numbers, unlike years that tend to be expressed with four digits. However, date conventions vary across the world.  For example, the following date might be interpreted differently depending on the country; it may refer to Saint Nicholas Day in 1929 or Anne Frank's birthday:

In [4]:
pd.to_datetime('12.6.1929')

Timestamp('1929-12-06 00:00:00')

To clarify towards Pandas that the first number refers to the day, you can add the parameter `dayfirst`:

In [5]:
pd.to_datetime('12.6.1929', dayfirst=True)

Timestamp('1929-06-12 00:00:00')

The method `to_datetime()` can also handle an array of date strings; it will return a `DatetimeIndex`, which is crucial for temporal indexing with Pandas.

In [6]:
sessions=["2.4.2020", "9.4.2020", "16.4.2020", "23.4.2020", "7.5.2020", "14.5.2020", "28.5.2020", "4.6.2020", "11.6.2020", "25.6.2020", "2.7.2020", "9.7.2020"]
pd.to_datetime(sessions, dayfirst=True)

DatetimeIndex(['2020-04-02', '2020-04-09', '2020-04-16', '2020-04-23',
               '2020-05-07', '2020-05-14', '2020-05-28', '2020-06-04',
               '2020-06-11', '2020-06-25', '2020-07-02', '2020-07-09'],
              dtype='datetime64[ns]', freq=None)

If you want to make extra sure that the date/time string is parsed correctly and quickly, you can pass a fixed `format` for the date/time strings to be parsed:

In [7]:
pd.to_datetime('2020-05-07', format="%Y-%m-%d")

Timestamp('2020-05-07 00:00:00')

In [8]:
# pd.to_datetime('8.5.1945 23:01', format="")

✏️ *Uncomment above line and enter the correct format to parse this historic event (for parsing hours and minutes, see [list of format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes))!*

### Load time series data

In this tutorial we will be analyzing energy generation and use in Germany from 1 Jan 2015 until 30 Sep 2020.

First we will load the freely accessible time series data gathered by the OPSD project from their [time series webpage](https://data.open-power-system-data.org/time_series/2020-10-06) (the URL below is generated using the filter with a focus on data for Germany). The values in the data are in megawatt (MW). 

`read_csv()` has a convenient feature, which lets you specify the column containing date/time information.

In [9]:
# data downloaded from OPSD - see how to filter elements on this page:
# https://data.open-power-system-data.org/time_series/2020-10-06

url = "https://data.open-power-system-data.org/index.php?package=time_series&version=2020-10-06&action=customDownload&resource=3&filter%5BRegion%5D%5B%5D=DE&filter%5BVariable%5D%5B%5D=load_actual_entsoe_transparency&filter%5BVariable%5D%5B%5D=solar_generation_actual&filter%5BVariable%5D%5B%5D=wind_generation_actual&downloadCSV=Download+CSV"

opsd = pd.read_csv(url, parse_dates=['utc_timestamp']) # parse timestamp column

✏️ *Check out the structure and content of `opsd` — Hint: `info()` and `head()` are your friends*

As you can tell it's a bit unwieldy. For simplicity sake, we `drop()` one of the redundant timestamp columns…

In [10]:
opsd = opsd.drop(columns=["cet_cest_timestamp"])

… and, while we're at it, we rename the column labels to make them shorter:

In [11]:
opsd = opsd.rename({
        'utc_timestamp': 'datetime',
        'DE_load_actual_entsoe_transparency': 'load',
        'DE_solar_generation_actual': 'solar',
        'DE_wind_generation_actual': 'wind'
    }, axis=1)


The time parsing resulted in one hour to be included from 2014. This is because this dataset covers the years 2015-2020 according to German time, while the generic UTC time is one hour 'behind'. To avoid having this stray hour in the old year, we revert the timestamps back to Germany's timezone:

In [12]:
opsd["datetime"] = opsd["datetime"].dt.tz_convert("Europe/Berlin")
opsd

,datetime,load,solar,wind
0,2015-01-01 00:00:00+01:00,NaN,NaN,NaN
1,2015-01-01 01:00:00+01:00,41151.0,NaN,8852.0
2,2015-01-01 02:00:00+01:00,40135.0,NaN,9054.0
3,2015-01-01 03:00:00+01:00,39106.0,NaN,9070.0
4,2015-01-01 04:00:00+01:00,38765.0,NaN,9163.0
...,...,...,...,...
50396,2020-09-30 21:00:00+02:00,57559.0,0.0,10654.0
50397,2020-09-30 22:00:00+02:00,54108.0,0.0,11836.0
50398,2020-09-30 23:00:00+02:00,49845.0,0.0,12168.0
50399,2020-10-01 00:00:00+02:00,46886.0,0.0,12533.0


The data for 2020 is not complete and lacks the fourth quarter. Let's focus on the years 2015-2019 then. The following code block reduces the data to these complete years:

In [13]:
start_date = pd.to_datetime('2015-01-01').tz_localize('Europe/Berlin')
end_date = pd.to_datetime('2019-12-31').tz_localize('Europe/Berlin')

opsd = opsd[(opsd['datetime'] >= start_date) & (opsd['datetime'] <= end_date)]
opsd

,datetime,load,solar,wind
0,2015-01-01 00:00:00+01:00,NaN,NaN,NaN
1,2015-01-01 01:00:00+01:00,41151.0,NaN,8852.0
2,2015-01-01 02:00:00+01:00,40135.0,NaN,9054.0
3,2015-01-01 03:00:00+01:00,39106.0,NaN,9070.0
4,2015-01-01 04:00:00+01:00,38765.0,NaN,9163.0
...,...,...,...,...
43796,2019-12-30 20:00:00+01:00,53959.0,0.0,32323.0
43797,2019-12-30 21:00:00+01:00,51937.0,0.0,32395.0
43798,2019-12-30 22:00:00+01:00,50574.0,0.0,31439.0
43799,2019-12-30 23:00:00+01:00,47382.0,0.0,30727.0


Did you notice that solar energy appears to be zero at the beginning and at the end of the DataFrame? Take a look at the time and you'll know why.

In order to get a random data `sample()` there is a method of the same name, which gives us a random set of rows:

In [14]:
opsd.sample(10)

,datetime,load,solar,wind
30768,2018-07-06 01:00:00+02:00,47216.0,0.0,8268.0
28066,2018-03-15 10:00:00+01:00,71289.0,14872.0,27646.0
42863,2019-11-21 23:00:00+01:00,55873.0,0.0,8022.0
41990,2019-10-16 15:00:00+02:00,66284.0,9223.0,16891.0
23353,2017-08-31 02:00:00+02:00,43078.0,0.0,12420.0
2828,2015-04-28 21:00:00+02:00,57794.0,0.0,6701.0
29465,2018-05-12 18:00:00+02:00,51503.0,6755.0,7842.0
34977,2018-12-28 09:00:00+01:00,56152.0,1889.0,8879.0
13004,2016-06-25 21:00:00+02:00,46816.0,69.0,8501.0
27625,2018-02-25 01:00:00+01:00,47887.0,0.0,18500.0


Now, let's take a look at the data types and values contained in our DataFrame:

In [15]:
opsd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43801 entries, 0 to 43800
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype                        
---  ------    --------------  -----                        
 0   datetime  43801 non-null  datetime64[ns, Europe/Berlin]
 1   load      43800 non-null  float64                      
 2   solar     43697 non-null  float64                      
 3   wind      43726 non-null  float64                      
dtypes: datetime64[ns, Europe/Berlin](1), float64(3)
memory usage: 1.7 MB


The values in the columns `load`, `solar` and `wind` are provided in the unit megawatt (MW) as integers.

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame:

In [16]:
opsd = opsd.set_index("datetime")

When we run `df.info()` again, you will see that the DataFrame now has a `DatetimeIndex`:

In [17]:
opsd.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43801 entries, 2015-01-01 00:00:00+01:00 to 2019-12-31 00:00:00+01:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   load    43800 non-null  float64
 1   solar   43697 non-null  float64
 2   wind    43726 non-null  float64
dtypes: float64(3)
memory usage: 1.3 MB


The `DatetimeIndex` provides a few handy methods to extract temporal units such as months, days, week of the year, etc.: 

In [18]:
opsd.index.year.unique()

Int64Index([2015, 2016, 2017, 2018, 2019], dtype='int64', name='datetime')

✏️ *Try to extract any other [temporal attributes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html):*


## 🕒 2. Process

The dataset contains over 40000 rows each of which contains data on overall energy load and renewable energy production. To make sense of all this data, we need to process the time series data into various chunks and sizes!


### Query time points and spans

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

One useful function of a temporal index, is its querying function. We can quickly extract the rows for a given time point or period.

Let's get the data for the day with the most daylight of 2017, i.e., the summer solstice or midsummer:

In [19]:
opsd.loc["2017-06-21"]

,load,solar,wind
datetime,,,
2017-06-21 00:00:00+02:00,47979.0,0.0,7394.0
2017-06-21 01:00:00+02:00,46039.0,0.0,6460.0
2017-06-21 02:00:00+02:00,44933.0,0.0,5894.0
2017-06-21 03:00:00+02:00,45097.0,0.0,5473.0
2017-06-21 04:00:00+02:00,45432.0,3.0,5007.0
2017-06-21 05:00:00+02:00,47851.0,303.0,4602.0
2017-06-21 06:00:00+02:00,55562.0,2095.0,3688.0
2017-06-21 07:00:00+02:00,61846.0,5926.0,2112.0
2017-06-21 08:00:00+02:00,65043.0,11143.0,1482.0


✏️ *In contrast, retrieve the data for the shortest day of 2017:*

The above query is an example of partial-string indexing: while our `DateTime` column actually contains time information as well, you can query it quickly (!) with just the date, or even a shorter query:

In [20]:
opsd.loc["2017-06"]

,load,solar,wind
datetime,,,
2017-06-01 00:00:00+02:00,46114.0,0.0,7902.0
2017-06-01 01:00:00+02:00,44291.0,0.0,7232.0
2017-06-01 02:00:00+02:00,43148.0,0.0,6642.0
2017-06-01 03:00:00+02:00,43231.0,0.0,6174.0
2017-06-01 04:00:00+02:00,43845.0,3.0,6159.0
...,...,...,...
2017-06-30 19:00:00+02:00,59780.0,3011.0,12645.0
2017-06-30 20:00:00+02:00,56144.0,876.0,10976.0
2017-06-30 21:00:00+02:00,53520.0,39.0,9854.0


You can also query a time period. Do you remember the storm Xavier that hit Germany in early October 2017? Let's retrieve the data around this time:

In [21]:
opsd.loc["2017-10-04":"2017-10-06"]

,load,solar,wind
datetime,,,
2017-10-04 00:00:00+02:00,42974.0,0.0,19096.0
2017-10-04 01:00:00+02:00,41380.0,0.0,19163.0
2017-10-04 02:00:00+02:00,41195.0,0.0,19163.0
2017-10-04 03:00:00+02:00,41717.0,0.0,19020.0
2017-10-04 04:00:00+02:00,43598.0,0.0,19456.0
...,...,...,...
2017-10-06 19:00:00+02:00,65541.0,0.0,21983.0
2017-10-06 20:00:00+02:00,61637.0,0.0,21763.0
2017-10-06 21:00:00+02:00,57880.0,0.0,21501.0


✏️ *Look up the energy data for a time period of your interest:*

### Aggregate values along time

The data we retrieved from OPSD, comes in the granularity of hours. To better understand the data it can be useful to reduce the data resolution and consider, for example, the total energy used/produced every day or the daily averages over the course of entire years.

The DataFrame's `resample()` method provides a concise and quick way of aggregating temporally indexed data along time units. Here we create a DataFrame with summed up values for each year aggregated from the original dataset:

In [22]:
sums = opsd.resample("Y").sum()
sums

,load,solar,wind
datetime,,,
2015-12-31 00:00:00+01:00,479454142.0,34796869.0,77163639.0
2016-12-31 00:00:00+01:00,486992473.0,34373377.0,75926639.0
2017-12-31 00:00:00+01:00,492119683.0,35882978.0,102654827.0
2018-12-31 00:00:00+01:00,498893806.0,41231973.0,108572938.0
2019-12-31 00:00:00+01:00,489392709.0,41878185.0,123399774.0


The resample operations can also be carried out one after another. For example, we might want to know how weekly energy use/production varies between the quarters:

In [23]:
# first we create the sums per week
weekly_sums = opsd.resample("W").sum()
# then we generate the weekly means for each quarter
quarterly_means = weekly_sums.resample("Q").mean()
# for readability we'll revert the values back to integers
quarterly_means.astype(int)

,load,solar,wind
datetime,,,
2015-03-31 00:00:00+02:00,9292593,349758,1559561
2015-06-30 00:00:00+02:00,8748509,1013038,1184655
2015-09-30 00:00:00+02:00,8956924,978015,1150870
2015-12-31 00:00:00+01:00,9523976,326500,1953362
2016-03-31 00:00:00+02:00,9692957,314387,1914033
2016-06-30 00:00:00+02:00,8942758,986066,1222605
2016-09-30 00:00:00+02:00,8895056,1046789,988603
2016-12-31 00:00:00+01:00,9738834,292048,1624855
2017-03-31 00:00:00+02:00,9963887,376470,2179402


✏️ *How did **daily** energy use/production change over the **years**?*

## 🥗 3. Present

Enough data processing. It's time for visualization!

### Time spans

One of the first time visualizations was [*A Chart of Biography*](https://en.wikipedia.org/wiki/A_Chart_of_Biography) (1765) by Joseph Priestley. Let's create a similar visualization of the US presidencies since World War II. First we load the CSV file with `pd.read_csv()`:

In [24]:
presidents = pd.read_csv("us_presidents.csv")
presidents

,name,start,end,party
0,Harry S. Truman,1945,1953,Democratic
1,Dwight D. Eisenhower,1953,1961,Republican
2,John F. Kennedy,1961,1963,Democratic
3,Lyndon B. Johnson,1963,1969,Democratic
4,Richard Nixon,1969,1974,Republican
5,Gerald Ford,1974,1977,Republican
6,Jimmy Carter,1977,1981,Democratic
7,Ronald Reagan,1981,1989,Republican
8,George H. W. Bush,1989,1993,Republican
9,Bill Clinton,1993,2001,Democratic


The start and end times of the presidencies are given as four-digit integers, i.e., years. To convey towards Pandas and Altair that the start and end columns are actually dates, we need to parse them using the `to_datetime()` method:

In [25]:
presidents['start'] = pd.to_datetime(presidents['start'], format="%Y")
presidents['end'] = pd.to_datetime(presidents['end'], format="%Y")

✏️ *Check what happens if the years are not parsed as DateTime values; to do this you can simply add a hash sign **#** in front of each of the two lines above, re-run above cell and the next one, too!*

The following chart consists of two parts: `bars` and `labels`. The former will be the main chart representing the time spans of the presidencies as bars, and the latter will add the presidents' names. This way we can position the labels right next to the bars, much nicer!

In [26]:
# we add height as a parameter to make the time spans appear less clunky
bars = alt.Chart(presidents).mark_bar(height=5).encode(
    # we use two columns for the x-positions where the bar starts and ends
    x='start',
    x2='end',
    # we sort by start dates and hide the axis, as names will be added below
    y=alt.Y('name').sort('x').axis(None),
    color="party"
)

# the labels are added to the chart using the mark_text command
# no need to specify x and y as this is re-used from the bar chart
labels = bars.mark_text(align='right', dx=-5).encode(
    text='name'
)

# both charts integrated into a layered chart through the magic of a plus sign
bars + labels

alt.LayerChart(...)

✏️ *Customize this chart. For example, you might want to change the colors associated with the parties… There was a time when orange has not been the color of the Republicans…*

### Overall trends

Next, we are going to return to the energy time series data that we prepared above. Remember that the original dataset has an hourly resolution resulting in too many data points to visualize at once. Altair itself handles at most 5,000 data rows.

To reduce the dataset into a manageable size, we will create daily sums with the `resample()` method of the DateTimeIndex:

In [27]:
days = opsd.resample("D").sum()

To visualize the contents of the DataFrame with Altair, we turn the date/time information in the index into its own column `datetime` again and transform the DataFrame from [wide to long form](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data) using the `melt()` method. We pass the name of the `datetime` column to the melt method so that it is treated as an index variable. To keep it short, these two steps `reset_index()` and `melt("datetime")` are done in one go right when we create the `Chart()`, which is what we are going to do in the rest of the tutorial. Here you see the dataframe in its long form:

In [28]:
opsd.reset_index().melt("datetime")

,datetime,variable,value
0,2015-01-01 00:00:00+01:00,load,NaN
1,2015-01-01 01:00:00+01:00,load,41151.0
2,2015-01-01 02:00:00+01:00,load,40135.0
3,2015-01-01 03:00:00+01:00,load,39106.0
4,2015-01-01 04:00:00+01:00,load,38765.0
...,...,...,...
131398,2019-12-30 20:00:00+01:00,wind,32323.0
131399,2019-12-30 21:00:00+01:00,wind,32395.0
131400,2019-12-30 22:00:00+01:00,wind,31439.0
131401,2019-12-30 23:00:00+01:00,wind,30727.0


Now let's start with a scatterplot visualization of the daily data - but before we will do, we will actually disable the maximum number of rows allowed, as Altair limites this to be only 5000 by default, and we have a few more:

In [29]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [30]:
alt.Chart(days.reset_index().melt("datetime")).mark_circle().encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=700, height=400)

alt.Chart(...)

With this our eyes can already see several patterns going on. Some are more dictinct than others. Interestingly, the load data (blue) seems to have two separate, but parallel curves…

✏️ *There is a lot of overplotting going on. Reduce the `size` and `opacity` of all dots, by passing these as parameters to `mark_circle`!*

Next, we are going to connect the dots and create a line chart form this data. So basically the same code as above, except we're now using `mark_line()` instead of `mark_circle`:

In [31]:
alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1).encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=700, height=400)

alt.Chart(...)

This chart already shows a lot: we can see weekly patterns—the jittery up and down—in the load (blue) and the seasonal patterns in the form of broad waves in the load and solar curves (blue and yellow). Right around the turn of the year we see a drop of energy load. What else do you see?

✏️ *Adjust the overall line width and opacity to help discern the solar and wind lines!*

While the above line chart is truthful to the local fluctuations, it makes it hard to actually grasp the ups and downs over the course of months and years. Let's change the sampling from days to months to examine the overall patterns in the data.

In [32]:
# we reduce the granularity to months and sum up the values
months = opsd.resample("M").sum()

# to make the line a bit smoother, we include an interpolate parameter
alt.Chart(months.reset_index().melt("datetime")).mark_line(opacity=0.75, interpolate="basis").encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=700, height=400)

alt.Chart(...)

What do you think? The fine-grained jitter is now gone and we might have lost too much detail. In fact, first downsampling the data and then including an interpolation is maybe giving it too much of a treatment (like overusing Photoshop's blur function). For example, the load minima around the turns of the year are not visible as the overall energy load in the winter months appears larger. On the other hand, the solar curve has become an almost perfect sine wave, probably relating to the position of the sun across the year.

✏️ *Play with different granularities in the `resample()` step. For example, use **W** for week or **Q** for quarter!*

One way to integrate the local and global patterns is to create a layered graph, as we have already done with the presidents' names above. This time we are combining a line chart of the days with a line chart of monthly averages.

Note how we ensure that both DataFrames fit the same data scale. As we're looking at energy values per day, we first generate the daily sums and on this basis the monthly means:

In [33]:
days = opsd.resample("D").sum()
months = days.resample("M").mean()

Next we create the line charts and combine the two again with the **+** operator:

In [34]:
chart1 = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1, opacity=0.25).encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=700, height=400) # not necessary to repeat below (the 2nd chart uses this)

chart2 = alt.Chart(months.reset_index().melt("datetime")).mark_line(interpolate="basis", opacity=1).encode(
    x='datetime',
    y='value',
    color='variable',
)

chart1 + chart2

alt.LayerChart(...)

With this view we already get a good sense of the overall time patterns, while still seeing some of the particular variations.

✏️ *Add the `.interactive()` directive to one of these charts to make them zoomable!*

### Rolling windows

While the `resample()` method takes a broad brush and results in a reduced dataset and a chart with smooth curves, `rolling()` offers an alternative way of smoothing out local outliers without actually reducing the resolution of the dataset. See [the Wikipedia article](https://en.wikipedia.org/wiki/Moving_average) for an in-depth explanation.

The first parameter determines the window size, the second one positions the window at the center (so that values are considered in both directions of the current date/time), and `win_type` determines how the values across the window are weighted; with the `triang` option added the values further away contribute less:

In [35]:
rolling = days.rolling(60, center=True, win_type="triang").mean()

chart1 = alt.Chart(rolling.reset_index().melt("datetime")).mark_line(strokeWidth=1.5, opacity=1).encode(
    x='datetime', y='value', color='variable',
).properties(width=700, height=400)

# same as the two charts in previous code cell, except more transparent
chart2 = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1, opacity=0.1).encode(
    x='datetime', y='value', color='variable',
)

chart3 = alt.Chart(months.reset_index().melt("datetime")).mark_line(interpolate="basis", opacity=.25).encode(
    x='datetime', y='value', color='variable',
)

chart1 + chart2 + chart3

alt.LayerChart(...)

Above you see the two lines for daily sums and monthly averages from the previous cell (slightly more transparent), on top of which you can see the time curve generated with a rolling window. It is quite apparent that this curve still features more pronounced dips around the end-of-year periods and elsewhere.


✏️ *Play around with different window sizes and other parameters in the first line in the cell above!*

### Detailed views

Some trends seem to pan out at much lower scales, which require higher levels of detail. For example, at the beginning of 2018 we see a peak in the wind energy production. If you remember, in the winter of 2017/2018 Germany and most of Europe actually witnessed several storms. And so it also happened in the first days of 2018, when [Burglind (a.k.a. Eleanor)](https://en.wikipedia.org/wiki/Storm_Eleanor_(2018)) passed by. Let's take a closer look at the first week of January 2018. For this we extract a subset of the DataFrame using our tried and tested `loc[]`, which lets us select specific time spans:

In [36]:
burglind = opsd.loc["2018-01-01":"2018-01-07"]

alt.Chart(burglind.reset_index().melt("datetime")).mark_line().encode(
    x='datetime',
    y='value',
    color='variable'
).properties(width=700, height=400)

alt.Chart(...)

✏️ *Are there other storms that you remember over the period from 2015 to 2019?*

### High-level bars

In contrast to the detailed close-up, we might also want to look at much more general patterns. Let's say we are curious about the total production of renewable energy over the five years in the dataset. 

For this we `resample` the original data from hourly data resolution into summed up values per year:

In [37]:
years = opsd.resample("Y").sum()

Next we create a new column, which contains the summed up values of solar and wind production.

In [38]:
years["renewable"] = years["solar"]+years["wind"]

We `drop` the other value columns:

In [39]:
renewable = years.drop(columns=["load", "solar", "wind"])

Extract the years and create a specific column:

In [40]:
renewable["year"] = years.index.year
renewable

,renewable,year
datetime,,
2015-12-31 00:00:00+01:00,111960508.0,2015
2016-12-31 00:00:00+01:00,110300016.0,2016
2017-12-31 00:00:00+01:00,138537805.0,2017
2018-12-31 00:00:00+01:00,149804911.0,2018
2019-12-31 00:00:00+01:00,165277959.0,2019


And we remove the datetime index, and this time discard it (i.e., `drop=True`), as we have the `year` column already:

In [41]:
renewable = renewable.reset_index(drop=True)
renewable

,renewable,year
0,111960508.0,2015
1,110300016.0,2016
2,138537805.0,2017
3,149804911.0,2018
4,165277959.0,2019


Now we can create a simple barchart from this:

In [42]:
alt.Chart(renewable).mark_bar(width=30, fill="lightgreen").encode(
    x='year:O',      # O ~ treat the years as ordinal data
    y='renewable:Q'  # Q ~ treat renewable as quantitative data
).properties(width=300, height=300)

alt.Chart(...)

✏️ *Rotate the chart into a horizontal bar chart! Note that you will also have to change a parameter of `mark_bar()`.*

### Cyclical patterns

Previously, we did notice a few recurring patterns in the energy load. One pattern might be due to the weekly work-and-leasure patterns. 

So let's take a closer look at the average energy `load` over the course of a week:


In [43]:
# remove solar and wind columns
weekdays = opsd.drop(columns=["solar", "wind"])

# add a column for weekdays
weekdays["weekday"] = weekdays.index.weekday

This time we do not use `resample()` to change the data granularity, but `groupby()` to study recurring data patterns, because we are actually interested in the energy load for each *generic* weekday across the entire dataset:

In [44]:
weekdays = weekdays.groupby("weekday").mean()

alt.Chart(weekdays.reset_index()).mark_bar(width=20).encode(
    x='weekday:O', # O ~ treat weekday as ordinal data
    y='load:Q'     # Q ~ treat load as quantitative data
).properties(width=300, height=300)

alt.Chart(...)

It's quite apparent that energy load is lower during weekends.

✏️ *Repeat this analysis of cyclical patterns for hourly energy load every generic day:*

### Interactive selection

If you want it all, a high-level overview and detailed close-ups, you might need two views that are coordinated:

In [45]:
weeks = opsd.resample("W").sum()
days = opsd.resample("D").sum()

brush = alt.selection_interval(encodings=['x'])

upper = alt.Chart(weeks.reset_index().melt("datetime")).mark_area(interpolate="basis").encode(
    x = alt.X('datetime:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=700, height=50).add_params(brush)

lower = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1).encode(
    x = alt.X('datetime:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=700, height=300)

upper & lower

alt.VConcatChart(...)

Note: The small stacked graph above the main chart is interactive, you can adjust the viewport of the line graph by dragging a time span within it.

✏️ *Add a comment for each line in above code cell and explain what is happening!*

## Sources

Tutorials & Examples
- [​​​​Tutorial: Time Series Analysis with Pandas by Jennifer Walker](https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/)
- [Altair Interval Selection Example](https://altair-viz.github.io/gallery/interval_selection.html)

Documentation: Pandas
- [Time series / date functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)
- [Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html)
- [DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html)

